In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam, Adadelta
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 20
compression = 0.8
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 41s 0us/step


In [0]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(x_train)


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 20
dropout_rate = 0.2
l = 20
input = Input(shape=(img_height, img_width, channel,))
#First_Conv2D = Conv2D(num_filter*2, (3,3), use_bias=False ,padding='same')(input)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 20)   540         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 20)   80          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 20)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

dropout_52 (Dropout)            (None, 8, 8, 16)     0           conv2d_53[0][0]                  
__________________________________________________________________________________________________
concatenate_50 (Concatenate)    (None, 8, 8, 176)    0           concatenate_49[0][0]             
                                                                 dropout_52[0][0]                 
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 8, 8, 176)    704         concatenate_50[0][0]             
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 8, 8, 176)    0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 (Conv2D)              (None, 8, 8, 16)     25344       activation_53[0][0]              
__________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adadelta(),
              metrics=['accuracy'])

In [0]:
filepath="latest_weight_ckpt_{epoch:02d}_{val_acc:.2f}.hdf5"
model_ckpt = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
lists = [model_ckpt]

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=lists)

#X_aug, y_aug = datagen.flow(x_train, y_train, batch_size=128)




'''
model.fit_generator(datagen.flow(x_train, y_train, batch_size=128), 
                    steps_per_epoch = x_train.shape[0]/batch_size,
                    epochs=epochs,
                    verbose = 1,
                    validation_data=(x_test, y_test),
                    callbacks=lists)
'''

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 667s 13ms/step - loss: 1.5751 - acc: 0.4272 - val_loss: 1.4991 - val_acc: 0.5029

Epoch 00001: val_acc improved from -inf to 0.50290, saving model to latest_weight_ckpt_01_0.50.hdf5
Epoch 2/50
13952/50000 [=======>......................] - ETA: 7:10 - loss: 1.1715 - acc: 0.5718

50000/50000 [==============================] - 640s 13ms/step - loss: 1.0804 - acc: 0.6097 - val_loss: 1.8991 - val_acc: 0.5033

Epoch 00002: val_acc improved from 0.50290 to 0.50330, saving model to latest_weight_ckpt_02_0.50.hdf5
Epoch 3/50
29440/50000 [================>.............] - ETA: 4:05 - loss: 0.9190 - acc: 0.6700

50000/50000 [==============================] - 640s 13ms/step - loss: 0.8925 - acc: 0.6805 - val_loss: 1.2848 - val_acc: 0.6037

Epoch 00003: val_acc improved from 0.50330 to 0.60370, saving model to latest_weight_ckpt_03_0.60.hdf5
Epoch 4/50
33408/50000 [===================>..........] - ETA: 3:17 - loss: 0.7971 - acc: 0.7149

50000/50000 [==============================] - 640s 13ms/step - loss: 0.7764 - acc: 0.7230 - val_loss: 1.2063 - val_acc: 0.6492

Epoch 00004: val_acc improved from 0.60370 to 0.64920, saving model to latest_weight_ckpt_04_0.65.hdf5
Epoch 5/50
34432/50000 [===================>..........] - ETA: 3:05 - loss: 0.6954 - acc: 0.7512

50000/50000 [==============================] - 641s 13ms/step - loss: 0.6919 - acc: 0.7546 - val_loss: 2.0889 - val_acc: 0.5076

Epoch 00005: val_acc did not improve from 0.64920
Epoch 6/50
41472/50000 [=======================>......] - ETA: 1:41 - loss: 0.6304 - acc: 0.7780

...

50000/50000 [==============================] - 639s 13ms/step - loss: 0.1011 - acc: 0.9635 - val_loss: 0.7244 - val_acc: 0.8551

Epoch 00047: val_acc did not improve from 0.86130
Epoch 48/50
44416/50000 [=========================>....] - ETA: 1:06 - loss: 0.0981 - acc: 0.9652

50000/50000 [==============================] - 640s 13ms/step - loss: 0.0993 - acc: 0.9648 - val_loss: 0.9216 - val_acc: 0.8296

Epoch 00048: val_acc did not improve from 0.86130
Epoch 49/50
44416/50000 [=========================>....] - ETA: 1:06 - loss: 0.0920 - acc: 0.9666

50000/50000 [==============================] - 638s 13ms/step - loss: 0.0919 - acc: 0.9667 - val_loss: 0.6719 - val_acc: 0.8674

Epoch 00049: val_acc improved from 0.86130 to 0.86740, saving model to latest_weight_ckpt_49_0.87.hdf5
Epoch 50/50
37120/50000 [=====================>........] - ETA: 2:33 - loss: 0.0935 - acc: 0.9659

50000/50000 [==============================] - 640s 13ms/step - loss: 0.0951 - acc: 0.9652 - val_loss: 0.8430 - val_acc: 0.8515

Epoch 00050: val_acc did not improve from 0.86740


'\nmodel.fit_generator(datagen.flow(x_train, y_train, batch_size=128), \n                    steps_per_epoch = x_train.shape[0]/batch_size,\n                    epochs=epochs,\n                    verbose = 1,\n                    validation_data=(x_test, y_test),\n                    callbacks=lists)\n'

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 53s 5ms/step
Test loss: 0.8430008523464203
Test accuracy: 0.8515


**Maximum validation accuracy  : 0.8674**

**Last epoch validation accuracy: 0.8515 **

In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')